In [1]:
import pandas as pd
import numpy as np
import json
import requests
import sys
from multiprocessing import cpu_count, Pool
sys.path.append('../')
from common import get_train, feat_eng,get_test, REF_PATH
from collections import defaultdict

In [2]:
with open('/Users/timlee/Dropbox/keys/yelp_api.txt') as f:
    client, api_key, secret   = f.read().strip().split(',') 

In [3]:
train_df = get_train()
train_df = feat_eng(train_df)

test_df = get_test()
test_df = feat_eng(test_df)

all_df = pd.concat([train_df, test_df])
print(all_df.shape)


uniq_coord = all_df.groupby(['From_lat','From_long'])[['pop']].count().reset_index()
uniq_coord.columns = ['lat','lon', 'ct']
uniq_coord['id'] = uniq_coord.index
from_uniq_coord = uniq_coord[['id','lat','lon']].copy()

uniq_coord = train_df.groupby(['To_lat','To_long'])[['pop']].count().reset_index()
uniq_coord.columns = ['lat','lon', 'ct']
uniq_coord['id'] = uniq_coord.index
to_uniq_coord = uniq_coord[['id','lat','lon']].copy()

Time it took: 0.14601588249206543
adding additional features to test and/or train ...

loading parking data 1.7M
loading address data 30K
calculating closest joins
merging
complete
fixing neighborhood string formatting
merging in neighborhood information
Time it took: 0.6889488697052002
Time it took: 0.084014892578125
adding additional features to test and/or train ...

loading parking data 1.7M
loading address data 30K
calculating closest joins
merging
complete
fixing neighborhood string formatting
merging in neighborhood information
Time it took: 0.4770698547363281
(1826, 49)


In [4]:
def start_job(gpslst):
    """
    generic parallelizing class
    """    
    cores = cpu_count() - 1 # for other processes

    data_split = np.array_split(gpslst, cores)
    pool = Pool(cores)
    pool.map(search_gps_list, data_split)
    pool.close()
    pool.join()

    
def search_gps_list(gpslst, reset=False):
    if reset:
        with open( REF_PATH + 'yelp_api_json.json', 'w') as f:
            f.write('')
            
    for idx, lat, lon in gpslst.values:
        yelp_search_coord(lat, lon, idx)

        
def yelp_search_coord(lat, lon, idx=0):

    REF_PATH = '/Users/timlee/myrepos/final-project-chengcheng-tim/ref_data/'
    
    biz_search = 'https://api.yelp.com/v3/businesses/search'
    
    data = {'grant_type': 'client_credentials',
            'client_id': client,
            'client_secret': secret}
    token = requests.post('https://api.yelp.com/oauth2/token', data=data)
    access_token = token.json()['access_token']
    headers = {'Authorization': 'bearer %s' % access_token}
    
    total_ct = 1
    rem_ct = total_ct
    index = 0
    while index < total_ct:
        
        params = {
            'location':'San Francisco'
            , 'latitude': lat #37.749044
            , 'longitude' : lon #-122.418166
            , 'radius': 500 # in meteres
            , 'limit' : 50
        }
        
        if index > 0:
            params['offset'] = index


        resp = requests.get(biz_search, headers=headers, params=params)
        biz_j = json.loads(resp.content)
        if biz_j.get('businesses') is None:
            break
        else:
            batch_ct = len(biz_j.get('businesses'))

            print(resp.status_code, idx, biz_j['total'], index )

            with open( REF_PATH + 'yelp_api_json.json', 'a') as f:
                out = {idx : biz_j}
                json.dump(out, f)
                f.write('\n')

            if index == 0:
                total_ct = biz_j['total']
                remaining_ct = total_ct - biz_j['total']
                index = batch_ct
            else:
                remaining_ct -= batch_ct
                index += batch_ct


In [5]:
search_gps_list(to_uniq_coord, True)

KeyError: 'access_token'

In [71]:
yelp_search_coord(37.749044, -122.418166)

200 0 833 0
200 0 833 50
200 0 833 100
200 0 833 150
200 0 833 200
200 0 833 250
200 0 833 300
200 0 833 350
200 0 833 400
200 0 833 450
200 0 833 500
200 0 833 550
200 0 833 600
200 0 833 650
200 0 833 700
200 0 833 750
200 0 833 800


## Reading in

In [36]:
json_per_line = []
with open(REF_PATH + 'yelp_api_json.json', 'r') as f:
    for line in f.readlines():
        json_per_line.append(json.loads(line.strip()))

In [72]:
biz_j['businesses']

[{'categories': [{'alias': 'mexican', 'title': 'Mexican'},
   {'alias': 'seafood', 'title': 'Seafood'},
   {'alias': 'sandwiches', 'title': 'Sandwiches'}],
  'coordinates': {'latitude': 37.75265, 'longitude': -122.41812},
  'display_phone': '(415) 824-7877',
  'distance': 401.419500708758,
  'id': 'el-farolito-san-francisco-2',
  'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/OPnKCvgBR2-lQ1-yahrpiA/o.jpg',
  'is_closed': False,
  'location': {'address1': '2779 Mission St',
   'address2': '',
   'address3': '',
   'city': 'San Francisco',
   'country': 'US',
   'display_address': ['2779 Mission St', 'San Francisco, CA 94110'],
   'state': 'CA',
   'zip_code': '94110'},
  'name': 'El Farolito',
  'phone': '+14158247877',
  'price': '$',
  'rating': 4.0,
  'review_count': 4303,
  'transactions': [],
  'url': 'https://www.yelp.com/biz/el-farolito-san-francisco-2?adjust_creative=JjtQ1txoSd69pSwUPsWxcA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=JjtQ1txoSd69p

6800

In [51]:
token.json()

{'access_token': 'e1eQ6CdDkrsPtzERZcm_YUHAyMBpl7XcSAgWFuvOrfofXjCygky4-AIICmDSRYpYqjtChJTJv2084_6gACmJko5vrW_TZiEcoTRlnC3BeoEdcXh6N6p2YbaLJI2HWnYx',
 'expires_in': 628649819,
 'token_type': 'Bearer'}